In [ ]:
# 🚀 Syft Code Queue - Quick Start

This is a minimal example to get you started with syft-code-queue in just a few minutes!


In [ ]:
# Step 1: Import and setup
import syft_code_queue as scq
from pathlib import Path
import tempfile

print("✅ Ready to use syft-code-queue!")


In [ ]:
# Step 2: Create simple code
code_dir = Path(tempfile.mkdtemp())

# Create a simple Python script
(code_dir / "hello.py").write_text('''
import os
print("Hello from remote execution!")
print(f"Job ID: {os.environ.get('SYFT_JOB_ID')}")

# Save a result
output_dir = os.environ.get('SYFT_OUTPUT_DIR', '.')
with open(f"{output_dir}/greeting.txt", "w") as f:
    f.write("Hello World from syft-code-queue!")
''')

# Create run.sh
(code_dir / "run.sh").write_text('''#!/bin/bash
echo "🚀 Starting hello job..."
python hello.py
echo "✅ Done!"
''')
(code_dir / "run.sh").chmod(0o755)

print(f"📦 Code created in: {code_dir}")


In [ ]:
# Step 3: Submit job
job = scq.submit_code(
    target_email="your-datasite@example.com",  # Change this!
    code_folder=code_dir,
    name="Hello World Job",
    tags=["demo", "hello-world"],
    auto_approval=True
)

print(f"🚀 Job submitted!")
print(f"📋 Job ID: {job.uid}")
print(f"🏷️ Status: {job.status.value}")


In [ ]:
# Step 4: Check results (when completed)
client = scq.create_client()
current_job = client.get_job(job.uid)

print(f"📊 Current status: {current_job.status.value}")

if current_job.status == scq.JobStatus.completed:
    # Get outputs
    output_path = client.get_job_output(job.uid)
    if output_path:
        greeting_file = output_path / "greeting.txt"
        if greeting_file.exists():
            print(f"📄 Result: {greeting_file.read_text()}")
    
    # Get logs
    logs = client.get_job_logs(job.uid)
    if logs:
        print(f"📋 Logs:\n{logs}")
else:
    print("⏳ Job still processing... check back later!")
